In [1]:
# import csv
import pandas as pd
import numpy as np
from collections import Counter
from itertools import islice
import langid
import re
# from ip2geotools.databases.noncommercial import DbIpCity

In [2]:
# 파싱된 eml 파일을 불러옴
input_file_name = 'C:/Users/Playdata/pytest/contest/eml_parse.csv'
eml = pd.read_csv(input_file_name, encoding= 'utf8')

C:\Users\Playdata\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (18,54,55,56,63,64,65,77,84,85,89,91,93) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
eml.head(5)

,file_name,MAIL_FROM,ORG_RCPT_TO,RCPT_TO,X-SPAM-TYPE,X-HELO,X-RECEIVED-IP,Received,X-MAIL-FROM,Message-ID,...,X-EN-SP-DIR,X-EN-SP-SQ,X-Accounttype,Mime-version,content-transfer-encoding,X-MS-TNEF-Correlator,text_without_tag,text_with_tag,include_url,num_of_imgs
0,059faa80-e6d3-47ff-8a45-edf0fba4e398.eml,juyoung@gmail.com,yunhan11@windows4unix.net,yunhan11@windows4unix.net,SPAM,helo 73.132.221.32,221.228.237.146,from [102.48.236.154] by 73.132.221.32 id <954...,juyoung@gmail.com,<l-$9342-$wxg3r73@1klb14lt57y>,...,NaN,NaN,NaN,NaN,NaN,NaN,Content-Type: text/html;\nContent-Transfer-Enc...,Content-Type: text/html;\nContent-Transfer-Enc...,0,0
1,059fd5ff-a546-4924-a6e9-a14f3bba805d.eml,suminy@outlook.kr,lgeman@dcmark.co.kr,lgeman@dcmark.co.kr,SPAM,helo 73.132.221.32,218.201.185.157,from [167.139.250.4] by 73.132.221.32 with ESM...,suminy@outlook.kr,<91yo0559041-z67@u8or7>,...,NaN,NaN,NaN,NaN,NaN,NaN,Content-Type: text/html;\nContent-Transfer-Enc...,Content-Type: text/html;\nContent-Transfer-Enc...,0,0
2,059fd98b-68f8-4722-8870-9671abfda544.eml,jisun@nate.com,jongchoo@meti.co.kr,jongchoo@meti.co.kr,SPAM,helo 73.132.221.32,180.167.196.58,from [211.204.37.3] by 73.132.221.32 with ESMT...,jisun@nate.com,<6$146$hq5oa018-kuc207kv@j3s.4.k.dgc>,...,NaN,NaN,NaN,NaN,NaN,NaN,Content-Type: text/html;\nContent-Transfer-Enc...,Content-Type: text/html;\nContent-Transfer-Enc...,0,0
3,059ffd38-f893-46ff-b65d-2f70da33dd90.eml,yenjuy@outlook.kr,tmdemhoy@dcmark.co.kr,tmdemhoy@dcmark.co.kr,SPAM,helo 73.132.221.32,122.97.133.171,from [208.206.157.152] by 73.132.221.32 with E...,yenjuy@outlook.kr,<f895$8$vvp51c869a5-$x9-s4@zqf.pb7w>,...,NaN,NaN,NaN,NaN,NaN,NaN,Content-Type: text/html;\nContent-Transfer-Enc...,Content-Type: text/html;\nContent-Transfer-Enc...,0,0
4,059ffe9c-b1a7-4b57-8174-8b0afe03366e.eml,noah@ajne.com,icon7447@skysilver.net,icon7447@skysilver.net,SPAM,ehlo ajne.com,42.248.200.68,from localhost ([127.0.0.1]) by localhost via ...,noah@ajne.com,<908A126CDA88BEF0D5EFDCB52CB5F03DCC16D3E1@YGLI...,...,NaN,NaN,NaN,NaN,NaN,NaN,This message contained an attachment that was ...,This message contained an attachment that was ...,0,0


In [16]:
# 불필요한 column들은 제거하고, 필요한 column만 가져와서 새로운 dataframe을 만듦
df = eml[['file_name','MAIL_FROM', 'Received', 'Date', 'text_without_tag', 'include_url', 'num_of_imgs']]

In [ ]:
# 'data' column에서 year 정보만 가져옴
temp = []
for i in df['Date']:
    if i is not np.nan:
        year = re.findall(r'20\d\d', i)
        if len(year) >0:
            temp.append(year[0])
        else:
            temp.append(np.nan)
    else:
        temp.append(np.nan)
df['Year'] = pd.Series( temp , index=df.index )

In [41]:
# 'data' column에서 month 정보만 가져옴
Months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', "Dec"]
temp = []

for i in df['Date']:
    is_empty = True
    
    if i is not np.nan:
        for j in Months:
            if j in i:
                temp.append(j)
                is_empty = False
                break

    if is_empty:
        temp.append(np.nan)


df['Month'] = pd.Series( temp , index=df.index )

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [44]:
# 'data' column에서 day(요일) 정보만 가져옴
Days = ['Mon', 'Tue' , 'Wed' , 'Thu', 'Fri', 'Sat', 'Sun']
temp = []

for i in df['Date']:
    is_empty = True
    
    if i is not np.nan:
        for j in Days:
            if j in i:
                temp.append(j)
                is_empty = False
                break

    if is_empty:
        temp.append(np.nan)


df['Day'] = pd.Series( temp , index=df.index )

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [62]:
# 'data' column에서 Time(시간) 정보만 가져옴
temp = []
for i in df['Date']:
    is_empty = True
    if i is not np.nan:
        p = i.find(':')
        if i[p-2:p].isdigit():
            temp.append(i[p-2:p])
            is_empty= False
    if is_empty:
        temp.append(np.nan )
        
df['Time'] = pd.Series( temp , index=df.index )

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [64]:
# 'text_without_tag'(본문) column에서 본문 길이 정보 가져옴

temp = []

for i in df['text_without_tag']:
    if i is not np.nan:
        temp.append(len(i))
    else:
        temp.append(np.nan )
        
df['length_of_text'] = pd.Series( temp , index=df.index )

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [72]:
# 'text_without_tag'(본문) column에서 작성 언어 정보 가져옴
temp = []

for i in df['text_without_tag']:
    if i is not np.nan:
        a = langid.classify(i)[0]
        temp.append(a)
    else:
        temp.append(np.nan)

df['main_language'] = pd.Series( temp , index=df.index )

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [75]:
# 'MAIL_FROM'(발신 아이디) column에서 발신 빈도에 따라 그룹을 나눔.
# 그룹 A : 만번 이상 발신 아이디 , 그룹 B : 천~만번 발신 아이디
# 그룹 C : 백~천번 발신 아이디.  그룹 D : 백번 이하 발신 아이디
counter = Counter(df['MAIL_FROM'])
sorted_id_freq = sorted([i for i in counter.values()]) 

In [84]:
# 발신자 아이디별로 그룹을 나누어 줌
A_list = [ i[0] for i in counter.items() if i[1] >= 10000 ]
B_list = [ i[0] for i in counter.items() if i[1] < 10000 and i[1] >= 1000 ]
C_list = [ i[0] for i in counter.items() if i[1] < 1000 and i[1] >= 100 ]
A_list.remove(np.nan)
# D_list = [ i[0] for i in counter.items if i[1] < quart1 ]

In [90]:
# 발신자의 해당 그룹에 맞는 그룹 정보를 부여함
temp = []

for i in df['MAIL_FROM']:
    if i is np.nan:
        temp.append(np.nan)
        continue
    if i in A_list:
        temp.append('A')
    elif i in B_list:
        temp.append('B')
    elif i in C_list:
        temp.append('C')
    else:
        temp.append('D')

df['Group'] = pd.Series( temp , index=df.index )

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [102]:
# 'Received' column에서 본문 ip 정보 가져옴
temp = []

for i in df['Received']:
    is_empty = True
    
    if i is not np.nan:
        ip = re.findall(r'([0-9]{1,3})\.([0-9]{1,3})\.([0-9]{1,3})\.([0-9]{1,3})', i)
        if len(ip) >0:
            com_ip = ip[0][0]+'.'+ip[0][1]+'.'+ip[0][2]+'.'+ip[0][3]
            temp.append(com_ip)
            is_empty = False
    # Received가 비어있거나, ip가 없을때 NaN처리
    if is_empty:
        temp.append(np.nan)
    
df['ip'] = pd.Series( temp , index=df.index )

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [104]:
# 새로 형성한 dataframe을 csv 형태로 저장
df.to_csv('C:/Users/Playdata/pytest/contest/main_df.csv', sep=',', index = False)